In [7]:
import pandas as pd
import glob
import os
import gc
from sklearn.utils import shuffle


pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

def merge(input1, input2, output):
    df = pd.read_csv(input1)
    df1 = pd.read_csv(input2)
    
    df = pd.concat([df, df1])
    
    df = df.sample(frac=1).reset_index(drop=True)
    
    df.to_csv(output, index=False)


def merge_from_parquet(folder_path, output):
    search_pattern = os.path.join(folder_path, '*.parquet')
    parquet_files = glob.glob(search_pattern)

    if not parquet_files:
        print(f"Lỗi: Không tìm thấy file .parquet nào trong '{folder_path}'.")
        return
    
    header = True
    for file in parquet_files:
        tmp_df = pd.read_parquet(file)
        tmp_df.to_csv(output, mode='a', index=False, header=header)
        header=False
        gc.collect()
    # print(f"Tìm thấy {len(parquet_files)} file parquet. Bắt đầu đọc và gộp...")
    # all_dfs = [pd.read_parquet(f) for f in parquet_files]
    # full_df = pd.concat(all_dfs, ignore_index=True)
    # print(f"Đã gộp thành công. Tổng số mẫu: {len(full_df)}")
    gc.collect()
    

def splitIL(input, output):
    search_pattern = os.path.join(input, '*.parquet')
    parquet_files = glob.glob(search_pattern)

    if not parquet_files:
        print(f"Lỗi: Không tìm thấy file .parquet nào trong '{input}'.")
        return
    
    all_dfs=[pd.read_parquet(f) for f in parquet_files]
    df = pd.concat(all_dfs, ignore_index=True)

    classes = sorted(df['label'].unique())
    # print(classes[:2])
    
    base_classes = classes[:1]
    new_classes = classes[1:]

    # config
    B_total = 100000
    N_new = 20000

    replay_per_class = lambda c: int(B_total / c)

    sample0= df[df['label'].isin([0,1])].groupby("label").apply(lambda x: x.sample(100000)).reset_index(drop=True)
    sample0 = shuffle(sample0)
    sample0.to_csv(output[0], index=False)
    
    del(sample0)
    gc.collect()
    
    for step, new_cls in enumerate(new_classes, start=1):
        old_cls = base_classes + new_classes[:step]
        # print("OLD CLASS: ", old_cls)
        E = replay_per_class(len(old_cls))
        
        old_samples = df[df["label"].isin(old_cls)].groupby("label").apply(lambda x: x.sample(E)).reset_index(drop=True)
        new_samples = df[df["label"] == new_cls].sample(N_new)
        
        incremental_df = shuffle(pd.concat([old_samples, new_samples]))
        incremental_df.to_csv(output[step], index=False)
        
        gc.collect()
    
input  = "C:/Users/hoang/Documents/Dataset_KLTN/scaled_output_parquet"

input1 = "C:/Users/hoang/Documents/Dataset_KLTN/training_dataset.csv"
input2 = "C:/Users/hoang/Documents/Dataset_KLTN/test_dataset.csv"
output = "C:/Users/hoang/Documents/Dataset_KLTN/merge.csv"
outputIL = ["session0.csv", "session1.csv", "session2.csv", "session3.csv"]

outputIL = ["C:/Users/hoang/Documents/Dataset_KLTN/"+ path for path in outputIL]

print(outputIL)
splitIL(input, outputIL)

gc.collect()

# df = pd.read_csv(output)
# print(df.head())


['C:/Users/hoang/Documents/Dataset_KLTN/session0.csv', 'C:/Users/hoang/Documents/Dataset_KLTN/session1.csv', 'C:/Users/hoang/Documents/Dataset_KLTN/session2.csv', 'C:/Users/hoang/Documents/Dataset_KLTN/session3.csv']


C:\Users\hoang\AppData\Local\Temp\ipykernel_6988\4058277735.py:66: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sample0= df[df['label'].isin([0,1])].groupby("label").apply(lambda x: x.sample(100000)).reset_index(drop=True)
C:\Users\hoang\AppData\Local\Temp\ipykernel_6988\4058277735.py:78: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  old_samples = df[df["label"].isin(old_cls)].groupby("label").apply(lambda x: x.samp

IndexError: list index out of range